In [1]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [18]:
import spacy
import re
import numpy as np


import pandas as pd
# Load and preprocess the text dataset
df = pd.read_excel('miniData.xlsx', engine='openpyxl')

# Convert the DataFrame to a CSV file with UTF-8 encoding
df.to_csv('DataPi.csv', index=False, encoding='utf-8')


# Load the dataset from the CSV file using Pandas
df = pd.read_csv('DataPi.csv')
# Remove whitespace from the 'Question' column
df['Question'] = df['                                                                                                             Question'].str.strip()
df.drop('                                                                                                             Question', axis=1, inplace=True)


nlp = spacy.load("en_core_web_sm")

questions = df['Question'].tolist()

keywords = []
for question in questions:
    doc = nlp(question)
    question_keywords = [token.text for token in doc if not token.is_stop and token.pos_ in ['NUM','NOUN', 'VERB', 'ADJ']]
    keywords.append(question_keywords)




keywords_str = [' '.join(k) for k in keywords]

def predict_formula(row):
    # Check if the Formula column already has a value
    
    
    # Check if both operands are present in the Answer column
    operands = re.findall(r"[\d.]+", keywords_str[row.name])
    print(operands)
    if len(operands) == 2:
        operand1 = float(operands[0])
        operand2 = float(operands[1])
        print(float(operands[0]))
        print(float(operands[1]))

        # Extract the operator from the question

        
def predict_operator(row):
    # Extract operands from the question
    operands = re.findall(r"[\d.]+", keywords_str[row.name])
    operand1 = int(operands[0])
    operand2 = int(operands[1])
    
    # Extract numerical answer from the Answer column
    answer = row["Answer"]
    
    # Predict operator
    operators = ["+", "-", "*", "/"]
    results = [eval(f"{operand1} {operator} {operand2}") for operator in operators]
    closest_result = min(results, key=lambda x: abs(x - answer))
    closest_operator = f"{operand1}{operators[np.argmin([abs(result - answer) for result in results])]}{operand2}"
    
    return closest_operator
def predict_operator1(row):
    # Extract operands from the question
    operands = re.findall(r"[\d.]+", keywords_str[row.name])
    operand1 = int(operands[0])
    operand2 = int(operands[1])
    
    # Extract numerical answer from the Answer column
    answer = row["Answer"]
    
    # Predict operator
    operators = ["+", "-", "*", "/"]
    results = [eval(f"{operand1} {operator} {operand2}") for operator in operators]
    closest_result = min(results, key=lambda x: abs(x - answer))
    closest_operator = f"{operators[np.argmin([abs(result - answer) for result in results])]}"
    
    return closest_operator
# Apply the predict_operator function to each row in the DataFrame
df["Formula"] = df.apply(predict_operator, axis=1)
df["Operator"] = df.apply(predict_operator1, axis=1)


# Select a random question from the dataset
random_row = df.sample(n=1).iloc[0]
question = random_row['Question']
operation = str(random_row['Operator'])
formula=str(random_row['Formula'])
# Print the question and its formula
print("Question:", question)
print("Operator:", operation)
print("Formula:", formula)

Question: John has 441 California. He did dried 62 Californias. How many Californias does he have now?
Operator: -
Formula: 441-62
